In [1]:
import numpy as np
import pandas as pd
import os
import re

# Load data
* Monthly player betting infomation

In [2]:
file_loc = 'C:/Users/markchen/Desktop/高胜率玩家分析/data/raw_data/'
file_name = '玩家投注资料_2020_Q1.csv'
raw_data = pd.read_csv(file_loc + file_name)

C:\Users\markchen\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (17) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


# Preprocessing

## Add game_type column
* 新增游戏种类栏位

In [3]:
game_type = raw_data.TournamentName_En.apply(lambda x: re.findall("\[(.*?)\]", x)).to_list()
game_type_update = [None if len(_) == 0 else _[0] for _ in game_type]
raw_data['Game_type'] = game_type_update
raw_data['Game_type'] = raw_data['Game_type'].replace({'LoL': 'LOL', 'AOV':'AoV'})
raw_data.fillna('Unknown', inplace = True)

# Groupby player
* 以玩家为单位，整合资讯

In [4]:
player_win_info = raw_data.groupby('MainAccountSN').agg(
    玩家总投注额 = ('RealBetAmount_CNY', 'sum'),
    玩家平均投注额 = ('RealBetAmount_CNY', 'mean'),
    玩家总损益 = ('CustomerWinLoss_CNY', 'sum'),
    玩家投注单量 = ('TransactionMGUID', 'count'),
    玩家季度获胜数 = ('BetResult', lambda x: sum(x == 'Won')),
    玩家季度胜率 = ('BetResult', lambda x: sum(x == 'Won') / len(x)),
    # 赔率资讯
    投注赔率平均 = ('InitailBetRate', 'mean'),
    投注赔率中位数 = ('InitailBetRate', 'median'),
    投注赔率标准差 = ('InitailBetRate', 'std'),
    # 游戏资讯
    投注游戏种类数 = ('Game_type', lambda x: x.nunique()),
    # 比赛资讯
    投注联赛数 = ('TournamentName_En', lambda x: x.nunique()),
    投注比赛数 = ('EventCode', lambda x: x.nunique()),
).reset_index()
player_win_info.fillna(0, inplace = True)

In [5]:
pd.options.display.float_format = "{:,.2f}".format
player_win_info

,MainAccountSN,玩家总投注额,玩家平均投注额,玩家总损益,玩家投注单量,玩家季度获胜数,玩家季度胜率,投注赔率平均,投注赔率中位数,投注赔率标准差,投注游戏种类数,投注联赛数,投注比赛数
0,13086,705.00,352.50,564.00,2,2,1.00,1.80,1.80,0.00,1,1,1
1,13124,"2,594.00",235.82,"-1,819.18",11,3,0.27,2.01,1.72,0.66,3,3,5
2,13201,100.00,100.00,-100.00,1,0,0.00,1.90,1.90,0.00,1,1,1
3,13275,"347,378.00","1,973.74","26,569.40",176,93,0.53,1.96,1.88,0.62,3,11,58
4,13283,280.00,56.00,13.28,5,2,0.40,2.25,2.27,0.45,1,1,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...
9846,9946041,300.00,300.00,99.00,1,1,1.00,1.33,1.33,0.00,1,1,1
9847,9946185,"1,750.00",134.62,"2,219.90",13,12,0.92,2.51,2.24,0.66,1,2,5
9848,9946916,"1,032.91",147.56,-71.07,7,3,0.43,2.09,1.93,0.55,2,2,2
9849,9947728,59.00,14.75,-6.40,4,3,0.75,1.96,1.31,1.30,1,2,3


## 注单量小于 Q1

In [ ]:
player_bet_lower_part_1 = player_win_info[player_win_info.玩家投注单量 <= player_win_info.玩家投注单量.quantile(0.25)].reset_index(drop = True)

In [ ]:
player_bet_lower_part_1.玩家总投注额.describe()

In [ ]:
player_bet_lower_part_1.sort_values('玩家总损益', ascending = False)

## 注单量 Q1 ~ Pr 95

### Create estimated true winning rate columns
* 新增依返还率推估真实机率栏位
* 新增相对胜率推估值
* 返还率: 0.92 , 0.95

In [53]:
q1_bet_num = player_win_info.玩家投注单量.quantile(0.25)
pr95_bet_num = player_win_info.玩家投注单量.quantile(0.95)
player_bet_lower_part_2 = player_win_info.query('@q1_bet_num <= 玩家投注单量 <= @pr95_bet_num').reset_index(drop = True)

In [54]:
# rr = 92
player_bet_lower_part_2['true_win_rate_92_mean'] = 1 / player_bet_lower_part_2['投注赔率平均'] * 0.92
player_bet_lower_part_2['true_win_rate_92_median'] = 1 / player_bet_lower_part_2['投注赔率中位数'] * 0.92
# rr = 95
player_bet_lower_part_2['true_win_rate_95_mean'] = 1 / player_bet_lower_part_2['投注赔率平均'] * 0.95
player_bet_lower_part_2['true_win_rate_95_median'] = 1 / player_bet_lower_part_2['投注赔率中位数'] * 0.95

In [55]:
col_names_list = list(player_bet_lower_part_2.filter(like = 'true_win_rate').columns)
col_names_list_provided = [_ + '_increment' for _ in col_names_list]
increment_df = player_bet_lower_part_2[['玩家季度胜率']].values / player_bet_lower_part_2.filter(like = 'true_win_rate') - 1
increment_df.columns = col_names_list_provided

In [56]:
player_bet_mid = pd.concat([player_bet_lower_part_2, increment_df], axis = 1)

In [57]:
player_bet_mid.投注赔率标准差.describe()

count   7,133.00
mean        0.88
std         0.99
min         0.00
25%         0.30
50%         0.60
75%         1.11
max        13.83
Name: 投注赔率标准差, dtype: float64

### Split df by odds std
* 将资料依玩家投注赔率标准差中位数分割

In [58]:
q2_odd_std = player_bet_mid.投注赔率标准差.quantile(0.5)
player_bet_mid_std_high = player_bet_mid.query('@q2_odd_std <= 投注赔率标准差').reset_index(drop = True)
player_bet_mid_std_low = player_bet_mid.query('投注赔率标准差 < @q2_odd_std').reset_index(drop = True)

### Calculate summary statistics
* 玩家投注单量中位数
* 投注比赛数中位数

In [67]:
bet_num_mid_std_low = player_bet_mid_std_low.玩家投注单量.quantile(0.5)
game_num_mid_std_low = player_bet_mid_std_low.投注比赛数.quantile(0.5)
target_index = player_bet_mid_std_low.true_win_rate_95_mean_increment.quantile(0.95)

In [68]:
# Low variance
player_bet_mid_std_low.query('玩家投注单量 > @bet_num_mid_std_low and \
                              投注比赛数 >= @game_num_mid_std_low and \
                              true_win_rate_95_mean_increment >= @target_index')

,MainAccountSN,玩家总投注额,玩家平均投注额,玩家总损益,玩家投注单量,玩家季度获胜数,玩家季度胜率,投注赔率平均,投注赔率中位数,投注赔率标准差,...,投注联赛数,投注比赛数,true_win_rate_92_mean,true_win_rate_92_median,true_win_rate_95_mean,true_win_rate_95_median,true_win_rate_92_mean_increment,true_win_rate_92_median_increment,true_win_rate_95_mean_increment,true_win_rate_95_median_increment
35,545065,"17,340.00",394.09,"18,099.00",44,34,0.77,2.68,2.75,0.54,...,5,9,0.34,0.33,0.35,0.35,1.25,1.31,1.18,1.24
1435,6534514,"9,611.36",533.96,"7,404.63",18,14,0.78,2.28,2.24,0.51,...,10,14,0.40,0.41,0.42,0.42,0.93,0.89,0.87,0.83
2273,8491244,"1,400.00",87.50,"1,547.78",16,13,0.81,2.36,2.35,0.38,...,1,11,0.39,0.39,0.40,0.40,1.08,1.08,1.02,1.01
2958,9389921,"7,894.00",375.90,"6,865.90",21,20,0.95,1.95,1.80,0.32,...,2,7,0.47,0.51,0.49,0.53,1.02,0.86,0.96,0.80
3508,9909096,"49,424.00","1,372.89","21,704.44",36,30,0.83,2.10,2.35,0.56,...,7,9,0.44,0.39,0.45,0.40,0.90,1.13,0.84,1.06


In [ ]:
# High variance

In [69]:
bet_num_mid_std_high = player_bet_mid_std_high.玩家投注单量.quantile(0.5)
game_num_mid_std_high = player_bet_mid_std_high.投注比赛数.quantile(0.5)
target_index_high = player_bet_mid_std_high.true_win_rate_95_median_increment.quantile(0.95)

In [70]:
player_bet_mid_std_high.query('玩家投注单量 > @bet_num_mid_std_high and \
                              投注比赛数 >= @game_num_mid_std_high and \
                              true_win_rate_95_median_increment >= @target_index')

,MainAccountSN,玩家总投注额,玩家平均投注额,玩家总损益,玩家投注单量,玩家季度获胜数,玩家季度胜率,投注赔率平均,投注赔率中位数,投注赔率标准差,...,投注联赛数,投注比赛数,true_win_rate_92_mean,true_win_rate_92_median,true_win_rate_95_mean,true_win_rate_95_median,true_win_rate_92_mean_increment,true_win_rate_92_median_increment,true_win_rate_95_mean_increment,true_win_rate_95_median_increment
1771,7313932,"134,127.00",798.38,"87,335.68",168,84,0.50,4.79,3.85,3.20,...,10,71,0.19,0.24,0.20,0.25,1.60,1.09,1.52,1.03
2660,8841618,"4,008.00",48.29,"6,507.26",83,45,0.54,4.39,3.45,2.86,...,6,26,0.21,0.27,0.22,0.28,1.59,1.03,1.50,0.97


## 注单量 > PR 95

In [71]:
pr95_bet_num = player_win_info.玩家投注单量.quantile(0.95)
player_bet_lower_part_3 = player_win_info.query('@pr95_bet_num < 玩家投注单量 ').reset_index(drop = True)

In [82]:
# rr = 93
player_bet_lower_part_3['true_win_rate_93_mean'] = 1 / player_bet_lower_part_3['投注赔率平均'] * 0.93
player_bet_lower_part_3['true_win_rate_93_median'] = 1 / player_bet_lower_part_3['投注赔率中位数'] * 0.93
# rr = 95
player_bet_lower_part_3['true_win_rate_95_mean'] = 1 / player_bet_lower_part_3['投注赔率平均'] * 0.95
player_bet_lower_part_3['true_win_rate_95_median'] = 1 / player_bet_lower_part_3['投注赔率中位数'] * 0.95

In [83]:
col_names_list = list(player_bet_lower_part_3.filter(like = 'true_win_rate').columns)
col_names_list_provided = [_ + '_increment' for _ in col_names_list]
increment_df = player_bet_lower_part_3[['玩家季度胜率']].values / player_bet_lower_part_3.filter(like = 'true_win_rate') - 1
increment_df.columns = col_names_list_provided

In [84]:
player_bet_high = pd.concat([player_bet_lower_part_3, increment_df], axis = 1)

### Split df by odds std
* 将资料依玩家投注赔率中位数分割

In [86]:
q2_odd_std = player_bet_high.投注赔率标准差.quantile(0.5)
player_bet_high_std_high = player_bet_high.query('@q2_odd_std <= 投注赔率标准差').reset_index(drop = True)
player_bet_high_std_low = player_bet_high.query('投注赔率标准差 < @q2_odd_std').reset_index(drop = True)

### Calculate summary statistics
* 玩家投注单量中位数
* 投注比赛数中位数

In [87]:
bet_num_high_std_low = player_bet_high_std_low.玩家投注单量.quantile(0.5)
game_num_high_std_low = player_bet_high_std_low.投注比赛数.quantile(0.5)
target_index = player_bet_high_std_low.true_win_rate_95_mean_increment.quantile(0.95)

In [88]:
# Low variance
player_bet_high_std_low.query('玩家投注单量 > @bet_num_high_std_low and \
                              投注比赛数 >= @game_num_high_std_low and \
                              true_win_rate_95_mean_increment >= @target_index')

,MainAccountSN,玩家总投注额,玩家平均投注额,玩家总损益,玩家投注单量,玩家季度获胜数,玩家季度胜率,投注赔率平均,投注赔率中位数,投注赔率标准差,...,投注联赛数,投注比赛数,true_win_rate_93_mean,true_win_rate_93_median,true_win_rate_95_mean,true_win_rate_95_median,true_win_rate_93_mean_increment,true_win_rate_93_median_increment,true_win_rate_95_mean_increment,true_win_rate_95_median_increment
58,2331915,"4,939,296.73","2,265.73","395,642.98",2180,1386,0.64,1.84,1.80,0.70,...,44,295,0.51,0.52,0.52,0.53,0.26,0.23,0.23,0.20
65,2533693,"1,562,772.00","1,934.12","29,493.27",808,549,0.68,1.72,1.50,0.78,...,36,323,0.54,0.62,0.55,0.63,0.26,0.10,0.23,0.07
128,5002333,"1,466,108.00","1,042.75","16,943.26",1406,896,0.64,1.79,1.66,0.79,...,53,387,0.52,0.56,0.53,0.57,0.23,0.14,0.20,0.11
171,7094493,"2,575,991.41","1,701.45","173,964.42",1514,977,0.65,1.74,1.69,0.39,...,31,331,0.53,0.55,0.55,0.56,0.21,0.17,0.18,0.15
187,7725224,"565,579.75",390.06,"22,340.49",1450,1177,0.81,1.40,1.19,0.81,...,33,352,0.66,0.78,0.68,0.80,0.23,0.04,0.20,0.02


In [ ]:
# High variance

In [93]:
bet_num_high_std_high = player_bet_high_std_high.玩家投注单量.quantile(0.5)
game_num_high_std_high = player_bet_high_std_high.投注比赛数.quantile(0.5)
target_index_high = player_bet_high_std_high.true_win_rate_95_median_increment.quantile(0.95)

In [94]:
player_bet_high_std_high.query('玩家投注单量 > @bet_num_high_std_high and \
                              投注比赛数 >= @game_num_high_std_high and \
                              true_win_rate_95_median_increment >= @target_index')

,MainAccountSN,玩家总投注额,玩家平均投注额,玩家总损益,玩家投注单量,玩家季度获胜数,玩家季度胜率,投注赔率平均,投注赔率中位数,投注赔率标准差,...,投注联赛数,投注比赛数,true_win_rate_93_mean,true_win_rate_93_median,true_win_rate_95_mean,true_win_rate_95_median,true_win_rate_93_mean_increment,true_win_rate_93_median_increment,true_win_rate_95_mean_increment,true_win_rate_95_median_increment
60,2725510,"90,055.04",145.96,"7,557.12",617,125,0.20,7.08,6.00,4.20,...,58,393,0.13,0.15,0.13,0.16,0.54,0.31,0.51,0.28
68,3102358,"160,620.00",175.35,"24,019.89",916,508,0.55,2.36,2.06,1.20,...,30,278,0.39,0.45,0.40,0.46,0.41,0.23,0.38,0.20
